I feel I have exhausted my efforts with random forests(which is not true), so here I will try various implentation of gradient Boosting trees from sklearn, XGBoost, LightGBM with differet feature selection measures, include feature interactions, deal with outliers and may be finally if it requires we can stack models. 

In [34]:
!pip install tqdm

    100% |████████████████████████████████| 61kB 4.5MB/s 


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from tqdm import tqdm
%matplotlib inline
rs = 100

In [0]:
train = pd.read_csv('drive/workspace/data/merc/train.csv')
test = pd.read_csv('drive/workspace/data/merc/test.csv')

In [0]:
cat_feat = list(train.select_dtypes(include=['object']).columns)
num_feat = list(train.select_dtypes(include=[np.number]).drop(['y'], axis=1).columns)

In [16]:
len(cat_feat), len(num_feat)

(8, 369)

So here cat_feat contains all categorical features, and num_feat has all numerical features excluding the target y. 

## Utility functions

Some of the utility functions are same as from nb2 but still here I am writing them again. 

In [0]:
def val_r2_score(estimator, X, y, n_splits=5, n_repeats=10, random_state=rs):
  val_scores = np.array([])
  trn_scores = np.array([])
  for i in tqdm(range(n_repeats)):
    kf = KFold(n_splits, True, random_state=rs+i)
    for ti, vi in kf.split(X):
      X_trn, X_val = X.iloc[ti], X.iloc[vi]
      y_trn, y_val = y.iloc[ti], y.iloc[vi]
      estimator.fit(X_trn, y_trn)
      val_scores = np.append(val_scores, r2_score(y_val, estimator.predict(X_val)))
      trn_scores = np.append(trn_scores, r2_score(y_trn, estimator.predict(X_trn)))
  return (val_scores.reshape((n_repeats, n_splits)), trn_scores.reshape( n_repeats, n_splits))

In [0]:
def train_val_r2_score(estimator, X, y, n_splits=3, n_repeats=10, random_state=rs):
    y_val_pred = pd.DataFrame(data=np.zeros(shape=(X.shape[0])),index=X.index, columns=['y'])
    scores = np.array([])
    train_scores = np.array([])
    for i in tqdm(range(n_repeats)):
        kf = KFold(n_splits=n_splits, random_state=random_state+i, shuffle=True)
        y_train_pred = pd.DataFrame(data=np.zeros(shape=(X.shape[0])),index=X.index, columns=['y'])
        for train_index, val_index in kf.split(X):
            X_ohe_train, X_ohe_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            _=estimator.fit(X_ohe_train, y_train)
            y_val_pred.iloc[val_index] = pd.DataFrame(estimator.predict(X_ohe_val), index=val_index, columns=['y'])
            y_train_pred.iloc[train_index] += pd.DataFrame(estimator.predict(X_ohe_train), index=train_index, columns=['y'])
        scores = np.append(scores, r2_score(y, y_val_pred))
        train_scores = np.append(train_scores, r2_score(y, y_train_pred/(n_splits-1)))
    return scores, train_scores

## Scikit Learn

Let's start with sklearn's implementation of GBM. 

In [0]:
X = train.drop(['y'], axis=1)
X = pd.get_dummies(X, columns=cat_feat)
y = train['y']

In [84]:
gbr = GradientBoostingRegressor()
val_scores, trn_scores = train_val_r2_score(gbr, X, y)

100%|██████████| 10/10 [01:20<00:00,  8.07s/it]


In [55]:
print(val_scores.mean(), val_scores.std())
print (trn_scores.mean(), trn_scores.std())
print (val_scores)

(0.5592712508115831, 0.05244097706310926)
(0.6577529734988582, 0.010593274705071789)
[[0.52770343 0.50505212 0.5545412  0.6141021  0.56593822]
 [0.60448107 0.61669633 0.51661907 0.5503775  0.47292347]
 [0.6047506  0.60202299 0.57138764 0.54706123 0.47375107]
 [0.5818624  0.43133665 0.58951768 0.56070441 0.60799904]
 [0.58047576 0.61070046 0.59302227 0.61685635 0.44035624]
 [0.55522127 0.58194286 0.62588408 0.5726197  0.47735789]
 [0.60163942 0.47447391 0.63212972 0.57288287 0.50519909]
 [0.52208136 0.57322103 0.5623261  0.51713715 0.60830957]
 [0.45906274 0.53498321 0.59982736 0.63724457 0.59071208]
 [0.58256642 0.5470017  0.61531224 0.49681789 0.577369  ]]


I will train a full vanilla GBM model and submit to see how it fares on leaderboard compared our score here. 

In [65]:
X_test = pd.get_dummies(test, columns=cat_feat)
X_test.shape, X.shape

((4209, 570), (4209, 564))

So there are some columns(ie some level of categorical columns) that are in test in not in train. So for the columns only in train we add them to test and for columns only in test we drop them. 

In [66]:
col1 = [f for f in X_test.columns if f not in X.columns]
col2 = [f for f in X.columns if f not in X_test.columns]
len(col1), len(col2)

(16, 10)

In [67]:
# We drop col1 from X_test and add col2 to X_test
X_test = X_test.drop(col1, axis=1)
new_df = pd.DataFrame(np.zeros((X_test.shape[0],len(col2))), index=X_test.index, columns=col2)
X_test = pd.concat([X_test, new_df], axis=1)
X_test.shape, X.shape

((4209, 564), (4209, 564))

In [70]:
gbr = GradientBoostingRegressor()
gbr.fit(X,y)
y_pred = gbr.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [0]:
subm = np.stack((X_test.ID, y_pred), axis=1)
np.savetxt('drive/workspace/data/merc/subm18.csv', subm, delimiter=',', header='ID,y', comments='', fmt=['%d','%.6f'])

Public LB : 0.52614, PrivateLB : 0.52965

Next instead of one hot encoding, we will target_encode our categories and then tune the GBM parameters. 